In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#from pyspark.types import *

Criar - Iniciar sessão PySpark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Pyspark01") \
    .getOrCreate()

In [3]:
df = spark.read.csv('archive/wc2018-players.csv', header=True, inferSchema=True)

In [4]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



Verificar valores nulos

In [6]:
df.toPandas().isna().sum()

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [7]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight', 'Peso')\

df.show(5)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|  Selecao|Numero|Posicao|              Nome|Nascimento|Shirt Name|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
only showing top 5 rows



Verificar valores nulos

In [8]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome 0
Nascimento 0
Shirt Name 0
Time 0
Altura 0
Peso 0


In [9]:
df.columns

['Selecao',
 'Numero',
 'Posicao',
 'Nome',
 'Nascimento',
 'Shirt Name',
 'Time',
 'Altura',
 'Peso']

Selecionar colunas

In [10]:
df.select('Nome','Selecao').show(7)

+------------------+---------+
|              Nome|  Selecao|
+------------------+---------+
|TAGLIAFICO Nicolas|Argentina|
|    PAVON Cristian|Argentina|
|    LANZINI Manuel|Argentina|
|    SALVIO Eduardo|Argentina|
|      MESSI Lionel|Argentina|
|  ANSALDI Cristian|Argentina|
|      BIGLIA Lucas|Argentina|
+------------------+---------+
only showing top 7 rows



In [11]:
df.select(col('Time')).show(7)

+--------------------+
|                Time|
+--------------------+
|      AFC Ajax (NED)|
|CA Boca Juniors (...|
|West Ham United F...|
|    SL Benfica (POR)|
|  FC Barcelona (ESP)|
|     Torino FC (ITA)|
|      AC Milan (ITA)|
+--------------------+
only showing top 7 rows



In [12]:
df.select(df['Selecao']).show(3)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 3 rows



Selecionar coluna com ALIAS

In [13]:
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



Select utilizando split

In [14]:
df.select('Selecao nome altura'.split()).show(5)

+---------+------------------+------+
|  Selecao|              nome|altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



Filtrar DF

In [15]:
df.filter('Selecao = "Brazil"').show(3)

+-------+------+-------+-----------+----------+-----------+--------------------+------+----+
|Selecao|Numero|Posicao|       Nome|Nascimento| Shirt Name|                Time|Altura|Peso|
+-------+------+-------+-----------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|       FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|     TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
+-------+------+-------+-----------+----------+-----------+--------------------+------+----+
only showing top 3 rows



Filtrar com mais de uma condição

In [16]:
df.filter((col('Selecao') == "Brazil") & (col('Altura') == 169)).show(10)

+-------+------+-------+----+----------+----------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome|Nascimento|Shirt Name|                Time|Altura|Peso|
+-------+------+-------+----+----------+----------+--------------------+------+----+
| Brazil|    18|     MF|FRED|05.03.1993|      FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+----+----------+----------+--------------------+------+----+



In [17]:
df.filter('Selecao = "Brazil"').filter(col('Numero') > 10).show(5)

+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
|Selecao|Numero|Posicao|             Nome|Nascimento| Shirt Name|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
only showing top 5 rows



Filtro condicional "OU" = |

In [18]:
df.filter((col('Nome') == 'MESSI Lionel') | (col('Nome') == 'Fred')).show()

+---------+------+-------+------------+----------+----------+------------------+------+----+
|  Selecao|Numero|Posicao|        Nome|Nascimento|Shirt Name|              Time|Altura|Peso|
+---------+------+-------+------------+----------+----------+------------------+------+----+
|Argentina|    10|     FW|MESSI Lionel|24.06.1987|     MESSI|FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------+----------+----------+------------------+------+----+



Filtrar DF combinando & e | (and - Or)

In [19]:
df.filter((col('Selecao') == "Brazil") & (col('Posicao') == 'DF') | (col('Nome') == 'MESSI Lionel')).show(5)

+---------+------+-------+------------+----------+-----------+--------------------+------+----+
|  Selecao|Numero|Posicao|        Nome|Nascimento| Shirt Name|                Time|Altura|Peso|
+---------+------+-------+------------+----------+-----------+--------------------+------+----+
|Argentina|    10|     FW|MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil|    22|     DF|      FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
|   Brazil|     6|     DF| FILIPE LUIS|09.08.1985|FILIPE LUIS|Atletico Madrid (...|   182|  73|
|   Brazil|    13|     DF|  MARQUINHOS|14.05.1994| MARQUINHOS|Paris Saint-Germa...|   183|  75|
|   Brazil|     3|     DF|     MIRANDA|07.09.1984|    MIRANDA|FC Internazionale...|   186|  78|
+---------+------+-------+------------+----------+-----------+--------------------+------+----+
only showing top 5 rows



In [20]:
df.show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|  Selecao|Numero|Posicao|              Nome|Nascimento|Shirt Name|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
only showing top 3 rows



Criar uma coluna nova

In [22]:
df = df.withColumn('Ano', substring('Nascimento', -4,4))

In [23]:
df.show()

+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|              Nome|Nascimento|Shirt Name|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|  72|1987|
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|  73|1986|
|Argentina|     5|     MF|      BIGLIA Lucas|3

Concatenar uma coluna com a outra

In [30]:
df.withColumn('Selecao_numero', concat('Selecao', col('Numero'))).show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
|  Selecao|Numero|Posicao|              Nome|Nascimento|Shirt Name|                Time|Altura|Peso| Ano|Selecao_numero|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|    Argentina3|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|1996|   Argentina22|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|1993|   Argentina15|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
only showing top 3 rows



Concatenar uma coluna utilizando separador

In [27]:
df.withColumn('Selecao_numero', concat_ws(' - ', 'Selecao', 'Numero')).show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
|  Selecao|Numero|Posicao|              Nome|Nascimento|Shirt Name|                Time|Altura|Peso| Ano|Selecao_numero|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| Argentina - 3|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|1996|Argentina - 22|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|1993|Argentina - 15|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+----+--------------+
only showing top 3 rows



Alterar o tipo da coluna

In [31]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



In [43]:
from pyspark.sql.types import IntegerType

df = df.withColumn('Ano', col('Ano').cast(IntegerType()))

In [44]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



In [48]:
dia = udf(lambda data: data.split('.')[0])

In [49]:
df.withColumn('Dia', dia('Nascimento')).show()

Py4JJavaError: An error occurred while calling o231.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 42) (KMS-NB258 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:82)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:853)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartiti